Explore TwitterAPI
====

Based on this code: http://geduldig.github.io/TwitterAPI/faulttolerance.html

In [32]:
reset -fs

In [33]:
from itertools import islice
from time import sleep

from credentials import credentials
from ftfy import fix_text
from textblob import TextBlob
from TwitterAPI import TwitterAPI

In [34]:
creds = credentials.require(['access_token', 
                             'access_token_secret', 
                             'consumer_key',
                             'consumer_secret'])

api = TwitterAPI(creds.consumer_key,
                 creds.consumer_secret,
                 creds.access_token,
                 creds.access_token_secret)

In [35]:
def tweets(query):
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q':query}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield(item['text'])
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [38]:
# Grab some tweets

t_pizza = tweets(query='pizza')
list(islice(t_pizza, 5))

['Es broma me voy a ver fargo que hoy como pizza ademas de estar solo lo dejo caer ;)',
 'RT @CedriCommeLaBD: "L\'amour peut décevoir. \nLa pizza jamais."\n- Emile Zola',
 'RT @CommonWhiteGrls: when ur mom says no pizza rolls but u put them in the cart anyway https://t.co/eZZwbqhssH',
 'comendo pizza',
 '容疑者Xの献身やってたんかーい\n\n見たかったわーい\n\nガリレオ全部見直したーい']

In [37]:
# Stream tweets and tag the words as they go by
t_beer = tweets(query='beer')

for _ in range(5):
    blob = TextBlob(fix_text(next(t_beer)))
    print("Tweet: ", blob.raw)
    print("Part of Speech (POS) Tags:",)
    [print("\t", token) for token in blob.tags]
    print('#'*40)
    sleep(1)

Tweet:  @DrunkenFunGuy beer tastes the same
Part of Speech (POS) Tags:
	 ('@', 'JJ')
	 ('DrunkenFunGuy', 'NNP')
	 ('beer', 'NN')
	 ('tastes', 'VBZ')
	 ('the', 'DT')
	 ('same', 'JJ')
########################################
Tweet:  World of Beer Drink it Intern Application: https://t.co/IDoJTgOcCW via @YouTube
Part of Speech (POS) Tags:
	 ('World', 'NNP')
	 ('of', 'IN')
	 ('Beer', 'NNP')
	 ('Drink', 'NNP')
	 ('it', 'PRP')
	 ('Intern', 'NNP')
	 ('Application', 'NNP')
	 ('https', 'NN')
	 ('//t.co/IDoJTgOcCW', 'JJ')
	 ('via', 'IN')
	 ('@', 'NN')
	 ('YouTube', 'NN')
########################################
Tweet:  Or whenever I was playing beer pong and I spilt a cup and yelled "my water just broke" 😂😂
Part of Speech (POS) Tags:
	 ('Or', 'CC')
	 ('whenever', 'WRB')
	 ('I', 'PRP')
	 ('was', 'VBD')
	 ('playing', 'VBG')
	 ('beer', 'NN')
	 ('pong', 'NN')
	 ('and', 'CC')
	 ('I', 'PRP')
	 ('spilt', 'VBP')
	 ('a', 'DT')
	 ('cup', 'NN')
	 ('and', 'CC')
	 ('yelled', 'VBD')
	 ('my', 'PRP$')
	 ('water

---
__The internet also has Rainbow Butterfly Unicorn Kittens:__

![](https://s-media-cache-ak0.pinimg.com/736x/bf/3f/4c/bf3f4c4e4cbc909f957f939bb6bc7cc6.jpg)